In [1]:
%pwd

'/Users/aakanksha/My_Codes/Deep-Learning_Kidney-Disease-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool
    params_epochs: int

In [3]:
from src.constants import *
from src.utils.common import read_yaml, create_directories
import os

class ConfigurationManager:
    
    def __init__(self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_train_model_config(self)->TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")

        create_directories([Path(training.root_dir)])

        training_model_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data = Path(training_data),
            params_batch_size = params.BATCH_SIZE,
            params_image_size = params.IMAGE_SIZE,
            params_is_augmentation = params.AUGMENTATION,
            params_epochs = params.EPOCHS
            ) 
        return training_model_config    


In [4]:
import tensorflow as tf
from src.utils.common import save_model

In [5]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)   

    def train_valid_generator(self):
        
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )    

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = True,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                shear_range = 0.2,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True,
                **data_generator_kwargs
            )

        else:
            train_data_generator = valid_data_generator

        self.train_generator = train_data_generator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'training',
            shuffle = True,
            **dataflow_kwargs
        )    

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
            )

        save_model(path=self.config.trained_model_path,
                model = self.model)
            


In [6]:
try:
    config = ConfigurationManager()
    training_config = config.get_train_model_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e    

[2023-10-20 22:23:47,398: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-20 22:23:47,401: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-20 22:23:47,401: INFO: common: created directory at: artifacts]
[2023-10-20 22:23:47,402: INFO: common: created directory at: artifacts/training]


2023-10-20 22:23:47.452725: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-10-20 22:23:47.452753: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-10-20 22:23:47.452768: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-10-20 22:23:47.453308: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-20 22:23:47.453723: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[2023-10-20 22:23:49,884: WARNING: optimizer: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/20


2023-10-20 22:23:50.762579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2023-10-20 22:23:50.802380: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


11/11 [==============================] - 25s 2s/step - loss: 13.2579 - accuracy: 0.5294 - val_loss: 7.6051 - val_accuracy: 0.5312
Epoch 2/20
11/11 [==============================] - 17s 2s/step - loss: 4.4119 - accuracy: 0.6912 - val_loss: 17.7403 - val_accuracy: 0.5156
Epoch 3/20
11/11 [==============================] - 19s 2s/step - loss: 8.8367 - accuracy: 0.6235 - val_loss: 8.6138 - val_accuracy: 0.4844
Epoch 4/20
11/11 [==============================] - 17s 2s/step - loss: 3.0461 - accuracy: 0.7471 - val_loss: 18.5176 - val_accuracy: 0.4688
Epoch 5/20
11/11 [==============================] - 17s 2s/step - loss: 4.9163 - accuracy: 0.7294 - val_loss: 8.3775 - val_accuracy: 0.5938
Epoch 6/20
11/11 [==============================] - 18s 2s/step - loss: 1.9074 - accuracy: 0.7941 - val_loss: 18.2829 - val_accuracy: 0.5000
Epoch 7/20
11/11 [==============================] - 17s 2s/step - loss: 8.3840 - accuracy: 0.6647 - val_loss: 5.1165 - val_accuracy: 0.5312
Epoch 8/20
11/11 [=========

/Users/aakanksha/miniforge3/envs/coffeeprogrammer/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
